In [44]:
# from triangle import Triangle as tri
import triangle as tr
# from paidincurredchain import PaidIncurredChain as PIC
import paidincurredchain as PIC

In [45]:
from importlib import reload
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

import pandas as pd
import numpy as np
from numpy.linalg import lstsq
import utils
import plotly.express as px
import plotly.graph_objects as go


from scipy.stats import probplot
import scipy.stats as stats

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_poisson_deviance, median_absolute_error, d2_tweedie_score
from sklearn.preprocessing import add_dummy_feature
from sklearn.model_selection import train_test_split, LeaveOneOut, TimeSeriesSplit, cross_val_predict, cross_val_score

import statsmodels as sm
import statsmodels.graphics.gofplots as smgof
import plotly.graph_objs as go
from scipy.stats import norm




%matplotlib inline

In [46]:
tr = reload(tr)
PIC = reload(PIC)

### load sample data from paper

In [47]:
rpt_tri, paid_tri = tr.Triangle.from_dahms()
rpt_tri.tri

,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1,3362115.0,5217243.0,4754900.0,4381677.0,4136883.0,4094140.0,4018736.0,3971591.0,3941391.0,3921258.0
2,2640443.0,4643860.0,3869954.0,3248558.0,3102002.0,3019980.0,2976064.0,2946941.0,2919955.0,NaN
3,2879697.0,4785531.0,4045448.0,3467822.0,3377540.0,3341934.0,3283928.0,3257827.0,NaN,NaN
4,2933345.0,5299146.0,4451963.0,3700809.0,3553391.0,3469505.0,3413921.0,NaN,NaN,NaN
5,2768181.0,4658933.0,3936455.0,3512735.0,3385129.0,3298998.0,NaN,NaN,NaN,NaN
6,3228439.0,5271304.0,4484946.0,3798384.0,3702427.0,NaN,NaN,NaN,NaN,NaN
7,2927033.0,5067768.0,4066526.0,3704113.0,NaN,NaN,NaN,NaN,NaN,NaN
8,3083429.0,4790944.0,4408097.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2761163.0,4132757.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
paid_tri.tri

,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1,1216632.0,1347072.0,1786877.0,2281606.0,2656224.0,2909307.0,3283388.0,3587549.0,3754403.0,3921258.0
2,798924.0,1051912.0,1215785.0,1349939.0,1655312.0,1926210.0,2132833.0,2287311.0,2567056.0,NaN
3,1115636.0,1387387.0,1930867.0,2177002.0,2513171.0,2931930.0,3047368.0,3182511.0,NaN,NaN
4,1052161.0,1321206.0,1700132.0,1971303.0,2298349.0,2645113.0,3003425.0,NaN,NaN,NaN
5,808864.0,1029523.0,1229626.0,1590338.0,1842662.0,2150351.0,NaN,NaN,NaN,NaN
6,1016862.0,1251420.0,1698052.0,2105143.0,2385339.0,NaN,NaN,NaN,NaN,NaN
7,948312.0,1108791.0,1315524.0,1487577.0,NaN,NaN,NaN,NaN,NaN,NaN
8,917530.0,1082426.0,1484405.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1001238.0,1376124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### load PIC model class & instantiate model

In [49]:
tr = reload(tr)
PIC=reload(PIC)

pic = PIC.PaidIncurredChain(paid_triangle=paid_tri, incurred_triangle=rpt_tri)

now we have a `PaidIncurredChain` model object that contains both paid and reported triangles, and the dataframes with the triangles can be accessed with the `.P` and `.I` attributes, respectively.

In [50]:
# sample incurred triangle
print(f"pic.I has shape {pic.I.shape} and type {type(pic.I)}")
pic.I

pic.I has shape (10, 10) and type <class 'pandas.core.frame.DataFrame'>


,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1,3362115.0,5217243.0,4754900.0,4381677.0,4136883.0,4094140.0,4018736.0,3971591.0,3941391.0,3921258.0
2,2640443.0,4643860.0,3869954.0,3248558.0,3102002.0,3019980.0,2976064.0,2946941.0,2919955.0,NaN
3,2879697.0,4785531.0,4045448.0,3467822.0,3377540.0,3341934.0,3283928.0,3257827.0,NaN,NaN
4,2933345.0,5299146.0,4451963.0,3700809.0,3553391.0,3469505.0,3413921.0,NaN,NaN,NaN
5,2768181.0,4658933.0,3936455.0,3512735.0,3385129.0,3298998.0,NaN,NaN,NaN,NaN
6,3228439.0,5271304.0,4484946.0,3798384.0,3702427.0,NaN,NaN,NaN,NaN,NaN
7,2927033.0,5067768.0,4066526.0,3704113.0,NaN,NaN,NaN,NaN,NaN,NaN
8,3083429.0,4790944.0,4408097.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2761163.0,4132757.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
# sample paid triangle
print(f"pic.P has shape {pic.P.shape} and type {type(pic.P)}")
pic.P

pic.P has shape (10, 10) and type <class 'pandas.core.frame.DataFrame'>


,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1,1216632.0,1347072.0,1786877.0,2281606.0,2656224.0,2909307.0,3283388.0,3587549.0,3754403.0,3921258.0
2,798924.0,1051912.0,1215785.0,1349939.0,1655312.0,1926210.0,2132833.0,2287311.0,2567056.0,NaN
3,1115636.0,1387387.0,1930867.0,2177002.0,2513171.0,2931930.0,3047368.0,3182511.0,NaN,NaN
4,1052161.0,1321206.0,1700132.0,1971303.0,2298349.0,2645113.0,3003425.0,NaN,NaN,NaN
5,808864.0,1029523.0,1229626.0,1590338.0,1842662.0,2150351.0,NaN,NaN,NaN,NaN
6,1016862.0,1251420.0,1698052.0,2105143.0,2385339.0,NaN,NaN,NaN,NaN,NaN
7,948312.0,1108791.0,1315524.0,1487577.0,NaN,NaN,NaN,NaN,NaN,NaN
8,917530.0,1082426.0,1484405.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1001238.0,1376124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


the triangle data that was passed to the model is really a data type called `Triangle` that is part of my `rocky` package.

the `Triangle` class is a subclass of `pandas.DataFrame` that has some extra methods and attributes that are useful for working with triangles. in particular it forces you to define origin periods and development periods, and it has methods for quickly reviewing the triangle data.

In [52]:
print(f"pic.P has type {type(pic.P)},\npic.paid_triangle has type {type(pic.paid_triangle)}")
pic.paid_triangle

pic.P has type <class 'pandas.core.frame.DataFrame'>,
pic.paid_triangle has type <class 'triangle.Triangle'>


           1          2          3          4          5          6    
ay                                                                     
1   1216632.0  1347072.0  1786877.0  2281606.0  2656224.0  2909307.0  \
2    798924.0  1051912.0  1215785.0  1349939.0  1655312.0  1926210.0   
3   1115636.0  1387387.0  1930867.0  2177002.0  2513171.0  2931930.0   
4   1052161.0  1321206.0  1700132.0  1971303.0  2298349.0  2645113.0   
5    808864.0  1029523.0  1229626.0  1590338.0  1842662.0  2150351.0   
6   1016862.0  1251420.0  1698052.0  2105143.0  2385339.0        NaN   
7    948312.0  1108791.0  1315524.0  1487577.0        NaN        NaN   
8    917530.0  1082426.0  1484405.0        NaN        NaN        NaN   
9   1001238.0  1376124.0        NaN        NaN        NaN        NaN   
10   841930.0        NaN        NaN        NaN        NaN        NaN   

           7          8          9          10  
ay                                              
1   3283388.0  3587549.0  3754403.0  

the triangles are stored as rocky `Triangle`s: this means you can use all the methods and attributes of the `Triangle` class on the model triangles to quickly get information about the triangles:

In [53]:
pic.incurred_triangle.ata_summary()

,1,2,3,4,5,6,7,8,9
1,1.552,0.911,0.922,0.944,0.99,0.982,0.988,0.992,0.995
2,1.759,0.833,0.839,0.955,0.974,0.985,0.99,0.991,
3,1.662,0.845,0.857,0.974,0.989,0.983,0.992,,
4,1.807,0.84,0.831,0.96,0.976,0.984,,,
5,1.683,0.845,0.892,0.964,0.975,,,,
6,1.633,0.851,0.847,0.975,,,,,
7,1.731,0.802,0.911,,,,,,
8,1.554,0.92,,,,,,,
9,1.497,,,,,,,,
Vol Wtd,,,,,,,,,


In [54]:
pic.paid_triangle.ata_summary()

,1,2,3,4,5,6,7,8,9
1,1.107,1.326,1.277,1.164,1.095,1.129,1.093,1.047,1.044
2,1.317,1.156,1.11,1.226,1.164,1.107,1.072,1.122,
3,1.244,1.392,1.127,1.154,1.167,1.039,1.044,,
4,1.256,1.287,1.159,1.166,1.151,1.135,,,
5,1.273,1.194,1.293,1.159,1.167,,,,
6,1.231,1.357,1.24,1.133,,,,,
7,1.169,1.186,1.131,,,,,,
8,1.18,1.371,,,,,,,
9,1.374,,,,,,,,
Vol Wtd,,,,,,,,,


In [55]:
print(f"Paid triangle has {pic.paid_triangle.n_rows} rows and {pic.paid_triangle.n_cols} columns")
pic.P

Paid triangle has 10 rows and 10 columns


,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1,1216632.0,1347072.0,1786877.0,2281606.0,2656224.0,2909307.0,3283388.0,3587549.0,3754403.0,3921258.0
2,798924.0,1051912.0,1215785.0,1349939.0,1655312.0,1926210.0,2132833.0,2287311.0,2567056.0,NaN
3,1115636.0,1387387.0,1930867.0,2177002.0,2513171.0,2931930.0,3047368.0,3182511.0,NaN,NaN
4,1052161.0,1321206.0,1700132.0,1971303.0,2298349.0,2645113.0,3003425.0,NaN,NaN,NaN
5,808864.0,1029523.0,1229626.0,1590338.0,1842662.0,2150351.0,NaN,NaN,NaN,NaN
6,1016862.0,1251420.0,1698052.0,2105143.0,2385339.0,NaN,NaN,NaN,NaN,NaN
7,948312.0,1108791.0,1315524.0,1487577.0,NaN,NaN,NaN,NaN,NaN,NaN
8,917530.0,1082426.0,1484405.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1001238.0,1376124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
# can access the underlying data frame with the .tri accessor:
pic.paid_triangle.tri

,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1,1216632.0,1347072.0,1786877.0,2281606.0,2656224.0,2909307.0,3283388.0,3587549.0,3754403.0,3921258.0
2,798924.0,1051912.0,1215785.0,1349939.0,1655312.0,1926210.0,2132833.0,2287311.0,2567056.0,NaN
3,1115636.0,1387387.0,1930867.0,2177002.0,2513171.0,2931930.0,3047368.0,3182511.0,NaN,NaN
4,1052161.0,1321206.0,1700132.0,1971303.0,2298349.0,2645113.0,3003425.0,NaN,NaN,NaN
5,808864.0,1029523.0,1229626.0,1590338.0,1842662.0,2150351.0,NaN,NaN,NaN,NaN
6,1016862.0,1251420.0,1698052.0,2105143.0,2385339.0,NaN,NaN,NaN,NaN,NaN
7,948312.0,1108791.0,1315524.0,1487577.0,NaN,NaN,NaN,NaN,NaN,NaN
8,917530.0,1082426.0,1484405.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1001238.0,1376124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# in PIC model this gets an alias/shortcut anyway:
pic.P

,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1,1216632.0,1347072.0,1786877.0,2281606.0,2656224.0,2909307.0,3283388.0,3587549.0,3754403.0,3921258.0
2,798924.0,1051912.0,1215785.0,1349939.0,1655312.0,1926210.0,2132833.0,2287311.0,2567056.0,NaN
3,1115636.0,1387387.0,1930867.0,2177002.0,2513171.0,2931930.0,3047368.0,3182511.0,NaN,NaN
4,1052161.0,1321206.0,1700132.0,1971303.0,2298349.0,2645113.0,3003425.0,NaN,NaN,NaN
5,808864.0,1029523.0,1229626.0,1590338.0,1842662.0,2150351.0,NaN,NaN,NaN,NaN
6,1016862.0,1251420.0,1698052.0,2105143.0,2385339.0,NaN,NaN,NaN,NaN,NaN
7,948312.0,1108791.0,1315524.0,1487577.0,NaN,NaN,NaN,NaN,NaN,NaN
8,917530.0,1082426.0,1484405.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1001238.0,1376124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
tr = reload(tr)
PIC=reload(PIC)

pic = PIC.PaidIncurredChain(paid_triangle=paid_tri, incurred_triangle=rpt_tri)

In [59]:
# can return the current diagonal with the .diag() method:
pic.paid_triangle.diag()

ay
1     3921258.0
2     2567056.0
3     3182511.0
4     3003425.0
5     2150351.0
6     2385339.0
7     1487577.0
8     1484405.0
9     1376124.0
10     841930.0
dtype: float64

In [60]:
# age to age factor triangle
pic.paid_triangle.ata()

,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1,1.107214,1.326490,1.276868,1.164190,1.095279,1.128581,1.092636,1.046509,1.044442,NaN
2,1.316661,1.155786,1.110344,1.226212,1.163654,1.107269,1.072429,1.122303,NaN,NaN
3,1.243584,1.391729,1.127474,1.154418,1.166626,1.039373,1.044347,NaN,NaN,NaN
4,1.255707,1.286803,1.159500,1.165903,1.150875,1.135462,NaN,NaN,NaN,NaN
5,1.272801,1.194365,1.293351,1.158661,1.166981,NaN,NaN,NaN,NaN,NaN
6,1.230668,1.356900,1.239740,1.133101,NaN,NaN,NaN,NaN,NaN,NaN
7,1.169226,1.186449,1.130787,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.179717,1.371369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.374422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
print("=======================================")
print("====== Paid Age-to-Age Factors ========")
print("========== .ata() method ==============")
print("=======================================")
print("")
pd.DataFrame({
    
    # all year VWA
    'Vol-Weighted Average': pic.paid_triangle.ata('vwa'),
    
    # VWA most recent 5 years
    'Vol-Weighted 5-Year': pic.paid_triangle.ata('vwa', 5),

    # simple ave most recent 2 years
    'Simple 2-Year': pic.paid_triangle.ata('simple', 2),

    # medial ave ex highest and lowest
    'Medial Ex. High/Low': pic.paid_triangle.ata('medial', excludes='hl'),
    }).round(3)

====== Paid Age-to-Age Factors ========
========== .ata() method ==============



,Vol-Weighted Average,Vol-Weighted 5-Year,Simple 2-Year,Medial Ex. High/Low
1,1.234,1.246,1.277,1.238
2,1.290,1.282,1.279,1.287
3,1.192,1.185,1.185,1.187
4,1.163,1.163,1.146,1.161
5,1.146,1.146,1.159,1.160
6,1.101,1.101,1.087,1.118
7,1.070,1.070,1.058,1.072
8,1.076,1.076,1.084,1.076
9,1.044,1.044,1.044,1.044
10,1.000,1.000,1.000,1.000


In [62]:
print("=======================================")
print("====== Paid Age-to-Ult Factors ========")
print("========== .atu() method ==============")
print("=======================================")
print("")

pd.DataFrame({
    
    # all year VWA
    'Vol-Weighted Average': pic.paid_triangle.atu('vwa'),
    
    # VWA most recent 5 years
    'Vol-Weighted 5-Year': pic.paid_triangle.atu('vwa', 5),

    # simple ave most recent 2 years
    'Simple 2-Year': pic.paid_triangle.atu('simple', 2),

    # medial ave ex highest and lowest
    'Medial Ex. High/Low': pic.paid_triangle.atu('medial', excludes='hl'),
    }).round(3)

====== Paid Age-to-Ult Factors ========
========== .atu() method ==============



,Vol-Weighted Average,Vol-Weighted 5-Year,Simple 2-Year,Medial Ex. High/Low
1,3.351,3.342,3.351,3.433
2,2.715,2.682,2.624,2.772
3,2.104,2.092,2.052,2.154
4,1.765,1.765,1.731,1.815
5,1.517,1.517,1.511,1.563
6,1.324,1.324,1.304,1.347
7,1.203,1.203,1.199,1.205
8,1.124,1.124,1.133,1.124
9,1.044,1.044,1.044,1.044
10,1.000,1.000,1.000,1.000


### now look at methods for the PIC model

In [66]:
# paid triangle (dataframe accessor)
pic.P

,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1,1216632.0,1347072.0,1786877.0,2281606.0,2656224.0,2909307.0,3283388.0,3587549.0,3754403.0,3921258.0
2,798924.0,1051912.0,1215785.0,1349939.0,1655312.0,1926210.0,2132833.0,2287311.0,2567056.0,NaN
3,1115636.0,1387387.0,1930867.0,2177002.0,2513171.0,2931930.0,3047368.0,3182511.0,NaN,NaN
4,1052161.0,1321206.0,1700132.0,1971303.0,2298349.0,2645113.0,3003425.0,NaN,NaN,NaN
5,808864.0,1029523.0,1229626.0,1590338.0,1842662.0,2150351.0,NaN,NaN,NaN,NaN
6,1016862.0,1251420.0,1698052.0,2105143.0,2385339.0,NaN,NaN,NaN,NaN,NaN
7,948312.0,1108791.0,1315524.0,1487577.0,NaN,NaN,NaN,NaN,NaN,NaN
8,917530.0,1082426.0,1484405.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1001238.0,1376124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# paid triangle (Triangle object)
pic.paid_triangle

           1          2          3          4          5          6    
ay                                                                     
1   1216632.0  1347072.0  1786877.0  2281606.0  2656224.0  2909307.0  \
2    798924.0  1051912.0  1215785.0  1349939.0  1655312.0  1926210.0   
3   1115636.0  1387387.0  1930867.0  2177002.0  2513171.0  2931930.0   
4   1052161.0  1321206.0  1700132.0  1971303.0  2298349.0  2645113.0   
5    808864.0  1029523.0  1229626.0  1590338.0  1842662.0  2150351.0   
6   1016862.0  1251420.0  1698052.0  2105143.0  2385339.0        NaN   
7    948312.0  1108791.0  1315524.0  1487577.0        NaN        NaN   
8    917530.0  1082426.0  1484405.0        NaN        NaN        NaN   
9   1001238.0  1376124.0        NaN        NaN        NaN        NaN   
10   841930.0        NaN        NaN        NaN        NaN        NaN   

           7          8          9          10  
ay                                              
1   3283388.0  3587549.0  3754403.0  

In [68]:
# age to age factor triangle (Triangle.ata() method)
pic.paid_triangle.ata()

,1,2,3,4,5,6,7,8,9,10
ay,,,,,,,,,,
1,1.107214,1.326490,1.276868,1.164190,1.095279,1.128581,1.092636,1.046509,1.044442,NaN
2,1.316661,1.155786,1.110344,1.226212,1.163654,1.107269,1.072429,1.122303,NaN,NaN
3,1.243584,1.391729,1.127474,1.154418,1.166626,1.039373,1.044347,NaN,NaN,NaN
4,1.255707,1.286803,1.159500,1.165903,1.150875,1.135462,NaN,NaN,NaN,NaN
5,1.272801,1.194365,1.293351,1.158661,1.166981,NaN,NaN,NaN,NaN,NaN
6,1.230668,1.356900,1.239740,1.133101,NaN,NaN,NaN,NaN,NaN,NaN
7,1.169226,1.186449,1.130787,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1.179717,1.371369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1.374422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


already have `Triangle.ata()` method to give the triangle of ata factors for a specific triangle, but `PaidIncurredChain._paid_ldf()` uses a slightly different display: 

In [71]:
# pic.paid_ldf()
pd.DataFrame(pic.paid_ldf())

,0,1,2,3,4,5,6,7,8,9
0,1216632.0,1.107214,1.326490,1.276868,1.164190,1.095279,1.128581,1.092636,1.046509,1.044442
1,798924.0,1.316661,1.155786,1.110344,1.226212,1.163654,1.107269,1.072429,1.122303,NaN
2,1115636.0,1.243584,1.391729,1.127474,1.154418,1.166626,1.039373,1.044347,NaN,NaN
3,1052161.0,1.255707,1.286803,1.159500,1.165903,1.150875,1.135462,NaN,NaN,NaN
4,808864.0,1.272801,1.194365,1.293351,1.158661,1.166981,NaN,NaN,NaN,NaN
5,1016862.0,1.230668,1.356900,1.239740,1.133101,NaN,NaN,NaN,NaN,NaN
6,948312.0,1.169226,1.186449,1.130787,NaN,NaN,NaN,NaN,NaN,NaN
7,917530.0,1.179717,1.371369,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1001238.0,1.374422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,841930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


here the first column is the first column of the paid triangle, and the second is the first ata factor column. 

this is how it is defined in the paper. 

### now compare paid LDF estimator from PIC to VWA

$$
% multi-line equation
\begin{align}
    \text{paid ldf estimate} \sim \text{LN}\left(\mu, s\right) \\
    \mu_j = \sum_{i=0}^{I-j} \log \frac{P_{i, j}}{P_{i, j-1}} \\
    s_j^2 = \frac{1}{I-j}\sum_{i=0}^{I-j} \left( \log \frac{P_{i, j}}{P_{i, j-1}} - \mu_j \right)^2
\end{align}

% \begin{equation}
    
% \end{equation}
$$

In [73]:
# from paper: log estimate of paid LDFs
print("PIC paid LDF estimator vs Standard Chain Ladder")
pd.DataFrame(dict(paper=stats.lognorm.mean(loc=pic.ldf_estimator('paid'), s=np.sqrt(pic.variance_estimator('paid')))
    , paper_se=stats.lognorm.std(loc=pic.ldf_estimator('paid'), s=np.sqrt(pic.variance_estimator('paid')))
    , chain_ladder=pic.paid_triangle.ata('vwa')[:-1])).round(3)

PIC paid LDF estimator vs Standard Chain Ladder


,paper,paper_se,chain_ladder
1,1.214,0.061,1.234
2,1.250,0.069,1.290
3,1.175,0.059,1.192
4,1.155,0.024,1.163
5,1.139,0.024,1.146
6,1.098,0.035,1.101
7,1.067,0.019,1.070
8,1.081,0.035,1.076
9,1.044,0.020,1.044


In [87]:
df = pd.DataFrame({
    # current diagonals using rocky.Triangles
    "Cum. Paid": (pic.paid_triangle.diag()/1000).astype(int)
    , 'CL Paid ATU':pic.paid_triangle.atu('vwa', 5, tail=1.05).round(3)
    , 'CL Paid Ultimate': (pic.paid_triangle.diag()/1000 * pic.paid_triangle.atu('vwa', 5, tail=1.05)).astype(int)
    , 'PIC Paid Ultimate': pic.posterior_ultimate_paid_loss() 
})

# df['CL Ultimate'] = (df['Cum. Paid'] * df['CL Paid ATU']).astype(int)
df

ValueError: operands could not be broadcast together with shapes (9,) (10,) 

In [74]:
print("PIC incurred LDF estimator vs Standard Chain Ladder")
pd.DataFrame(dict(
    paper=stats.lognorm.mean(loc=pic.ldf_estimator('incurred'), s=np.sqrt(pic.variance_estimator('incurred')))
    , paper_se=stats.lognorm.std(loc=pic.ldf_estimator('incurred'), s=np.sqrt(pic.variance_estimator('incurred')))
    , chain_ladder=pic.incurred_triangle.ata('vwa')[:-1])).round(3)

PIC incurred LDF estimator vs Standard Chain Ladder


,paper,paper_se,chain_ladder
1,1.503,0.060,1.650
2,0.845,0.043,0.856
3,0.862,0.038,0.872
4,0.961,0.011,0.961
5,0.981,0.007,0.981
6,0.983,0.001,0.983
7,0.990,0.002,0.990
8,0.992,0.001,0.992
9,0.995,0.000,0.995


PIC model uses priors if they are passed to the model, but if not it automatically calculates the priors from the data.

In [29]:
tr = reload(tr)
PIC = reload(PIC)
rpt_tri, paid_tri = tr.Triangle.from_dahms()
pic = PIC.PaidIncurredChain(paid_triangle=paid_tri, incurred_triangle=rpt_tri)

In [30]:
# use the .ParameterTable() method to access parameter estimates
# prior and posterior are shown, but posterior will be the same as prior unless you either run the MCMC process or pass in a prior
# that is not directly estimated from the data 
pic.ParameterTable('phi') # log paid loss mean

,parameter,age,prior,posterior
0,phi,1,0.212341,0.212341
1,phi,2,0.247452,0.247452
2,phi,3,0.173157,0.173157
3,phi,4,0.154212,0.154212
4,phi,5,0.138326,0.138326
5,phi,6,0.097129,0.097129
6,phi,7,0.067304,0.067304
7,phi,8,0.080421,0.080421
8,phi,9,0.043483,0.043483


In [31]:
pic.variance_estimator('paid')

array([0.00375492, 0.00467934, 0.00350932, 0.00058232, 0.00058546,
       0.00122724, 0.00034396, 0.0012223 , 0.00038262])

In [32]:
pic.ParameterTable('sigma2')

,parameter,age,prior,posterior
0,sigma2,1,0.003755,0.003755
1,sigma2,2,0.004679,0.004679
2,sigma2,3,0.003509,0.003509
3,sigma2,4,0.000582,0.000582
4,sigma2,5,0.000585,0.000585
5,sigma2,6,0.001227,0.001227
6,sigma2,7,0.000344,0.000344
7,sigma2,8,0.001222,0.001222
8,sigma2,9,0.000383,0.000383


In [33]:
pic.variance_estimator('incurred')

array([3.55983330e-03, 1.84330050e-03, 1.47864463e-03, 1.23161078e-04,
       5.48390022e-05, 2.18309079e-06, 2.43378655e-06, 6.13413672e-07,
       1.57244527e-07])

In [34]:
pic.ParameterTable('tau2')

,parameter,age,prior,posterior
0,tau2,1,3.559833e-03,3.559833e-03
1,tau2,2,1.843300e-03,1.843300e-03
2,tau2,3,1.478645e-03,1.478645e-03
3,tau2,4,1.231611e-04,1.231611e-04
4,tau2,5,5.483900e-05,5.483900e-05
5,tau2,6,2.183091e-06,2.183091e-06
6,tau2,7,2.433787e-06,2.433787e-06
7,tau2,8,6.134137e-07,6.134137e-07


In [35]:
# prior estimates of paid loss log LDFs
pic.ParameterTable('phi')

,parameter,age,prior,posterior
0,phi,1,0.212341,0.212341
1,phi,2,0.247452,0.247452
2,phi,3,0.173157,0.173157
3,phi,4,0.154212,0.154212
4,phi,5,0.138326,0.138326
5,phi,6,0.097129,0.097129
6,phi,7,0.067304,0.067304
7,phi,8,0.080421,0.080421
8,phi,9,0.043483,0.043483


In [36]:
# prior estimates of incurred loss log LDFs
pic.ParameterTable('psi')

,parameter,age,prior,posterior
0,psi,1,0.500786,0.500786
1,psi,2,-0.156344,-0.156344
2,psi,3,-0.138431,-0.138431
3,psi,4,-0.038878,-0.038878
4,psi,5,-0.019489,-0.019489
5,psi,6,-0.016724,-0.016724
6,psi,7,-0.009871,-0.009871
7,psi,8,-0.008416,-0.008416
8,psi,9,-0.005121,-0.005121


In [37]:
tr = reload(tr)
PIC = reload(PIC)
rpt_tri, paid_tri = tr.Triangle.from_dahms()
pic = PIC.PaidIncurredChain(
    paid_triangle=paid_tri
    , incurred_triangle=rpt_tri
    , prior_phi=np.array([0.3, 0.2, 0.16, 0.16, 0.13,
       0.1, 0.08 , 0.07, 0.04])
    , prior_s2=0.001
    )

In [38]:
pic.empirical_phi_mean()

array([0.21234112, 0.24745198, 0.17315712, 0.15421192, 0.13832647,
       0.09712865, 0.0673038 , 0.08042144, 0.04348324])

In [39]:
pic.posterior_phi_mean()

array([0.23464908, 0.23232569, 0.16946607, 0.15460465, 0.13768382,
       0.09761622, 0.06812099, 0.07798226, 0.04308923])

In [40]:
pic.posterior_phi_sample_variance()

array([2.54485948e-04, 3.18770413e-04, 2.80536280e-04, 6.78514360e-05,
       7.71814111e-05, 1.69807075e-04, 6.43645667e-05, 2.34053784e-04,
       1.13114212e-04])

In [41]:
tr = reload(tr)
PIC = reload(PIC)
rpt_tri, paid_tri = tr.Triangle.from_dahms()
pic = PIC.PaidIncurredChain(
    paid_triangle=paid_tri
    , incurred_triangle=rpt_tri
    )

In [42]:
pd.DataFrame(dict(j=range(9), sigma=np.sqrt(pic.sigma2), tau=np.sqrt(np.append(pic.tau2, 0 )))).set_index('j').transpose()

j,0,1,2,3,4,5,6,7,8
sigma,0.061277,0.068406,0.059239,0.024131,0.024196,0.035032,0.018546,0.034961,0.019561
tau,0.059664,0.042934,0.038453,0.011098,0.007405,0.001478,0.001560,0.000783,0.000000


In [43]:
pic.posterior_ultimate_paid_loss()

ValueError: operands could not be broadcast together with shapes (9,) (10,) 